In [12]:
import requests

In [ ]:
import webbrowser

In [1]:
import requests

In [13]:
url = 'http://ipaddress/ptz/11?action='

In [14]:
left = 'pan-left'

In [15]:
right = 'pan-right'

In [16]:
goleft = url + left

In [17]:
goright = url + right

In [ ]:
goleft

In [136]:
webbrowser.open(go)

True

In [19]:
session = requests.Session()
session.auth = ("username","password")

In [20]:
session.get(goright)

<Response [200]>

In [21]:
session.get(goleft)

<Response [200]>

In [23]:
import cv2

cap=cv2.VideoCapture('http://ipaddress/live/video12.m3u8?user=&password=')

while True:

    #print('About to start the Read command')
    ret, frame = cap.read()
    #print('About to show frame of Video.')
    cv2.imshow("Capturing",frame)
    #print('Running..')

    if cv2.waitKey(1) & 0xFF == ord('q'): session.get(goleft)
        
    if cv2.waitKey(1) & 0xFF == ord('w'): session.get(goright)
        
    if cv2.waitKey(1) & 0xFF == ord('e'):    
        break
    # ptz/0?action=pan-left&speed=50 HTTP/1.1

cap.release()
cv2.destroyAllWindows()

In [22]:
import cv2

# cv2.waitKey(delay=None) -> retval

while True:
    if ord('q'): session.get(goleft)
        
    if ord('w'): session.get(goright)
        
    if ord('e'):    
        break
    # ptz/0?action=pan-left&speed=50 HTTP/1.1

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
from time import sleep

from onvif import ONVIFCamera

XMAX = 1
XMIN = -1
YMAX = 1
YMIN = -1

def perform_move(ptz, request, timeout):
    # Start continuous move
    ptz.ContinuousMove(request)
    # Wait a certain time
    sleep(timeout)
    # Stop continuous move
    ptz.Stop({'ProfileToken': request.ProfileToken})

def move_up(ptz, request, timeout=1):
    print ('move up...')
    request.Velocity.PanTilt._x = 0
    request.Velocity.PanTilt._y = YMAX
    perform_move(ptz, request, timeout)

def move_down(ptz, request, timeout=1):
    print ('move down...')
    request.Velocity.PanTilt._x = 0
    request.Velocity.PanTilt._y = YMIN
    perform_move(ptz, request, timeout)

def move_right(ptz, request, timeout=1):
    print ('move right...')
    request.Velocity.PanTilt._x = XMAX
    request.Velocity.PanTilt._y = 0
    perform_move(ptz, request, timeout)

def move_left(ptz, request, timeout=1):
    print ('move left...')
    request.Velocity.PanTilt._x = XMIN
    request.Velocity.PanTilt._y = 0
    perform_move(ptz, request, timeout)

def continuous_move():
    mycam = ONVIFCamera(url, 80, 'username', 'password')
    # Create media service object
    media = mycam.create_media_service()
    # Create ptz service object
    ptz = mycam.create_ptz_service()

    # Get target profile
    media_profile = media.GetProfiles()[0];

    # Get PTZ configuration options for getting continuous move range
    request = ptz.create_type('GetConfigurationOptions')
    request.ConfigurationToken = media_profile.PTZConfiguration._token
    ptz_configuration_options = ptz.GetConfigurationOptions(request)

    request = ptz.create_type('ContinuousMove')
    request.ProfileToken = media_profile._token

    ptz.Stop({'ProfileToken': media_profile._token})

    # Get range of pan and tilt
    # NOTE: X and Y are velocity vector
    global XMAX, XMIN, YMAX, YMIN
    XMAX = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].XRange.Max
    XMIN = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].XRange.Min
    YMAX = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].YRange.Max
    YMIN = ptz_configuration_options.Spaces.ContinuousPanTiltVelocitySpace[0].YRange.Min

    # move right
    move_right(ptz, request)

    # move left
    move_left(ptz, request)

    # Move up
    move_up(ptz, request)

    # move down
    move_down(ptz, request)

if __name__ == '__main__':
    continuous_move()